In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from data import load_data

from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,Callback
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

from tensorflow.keras import utils
from tensorflow.keras import losses
from tensorflow.keras import backend as K

import sys

## Multi task learning DNN

In [2]:
def set_seed(s):
    K.clear_session()
    
    # Seed value
    # Apparently you may use different seed values at each stage
    seed_value= s

    # 1. Set `PYTHONHASHSEED` environment variable at a fixed value
    import os
    os.environ['PYTHONHASHSEED']=str(seed_value)

    # 2. Set `python` built-in pseudo-random generator at a fixed value
    import random
    random.seed(seed_value)

    # 3. Set `numpy` pseudo-random generator at a fixed value
    import numpy as np
    np.random.seed(seed_value)

    # 4. Set the `tensorflow` pseudo-random generator at a fixed value
    import tensorflow as tf
    tf.random.set_seed(seed_value)
    # for later versions: 
    # tf.compat.v1.set_random_seed(seed_value)

In [4]:
datapath ="../../data/task_2.csv"
train_folds = load_data(datapath)

/home/vsabando/virtualenvs/tf_gpu/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [5]:
#--------------------------------------------------------
# Algunas constantes
#--------------------------------------------------------

#dropout
prob_h1 = 0.25
prob_h2 = 0.15
prob_h3 = 0.1

#batch normalization
momentum_batch_norm = 0.9

#adam optimizador
l_rate = 0.0001

#optimizador
adam_opt = Adam(l_rate)

# activacion capas internas
act = 'relu'

#early stopping
min_delta_val = 0.0001
patience_val = 500

# n_epochs = np.iinfo(np.int32).max
n_epochs = 1


In [6]:
def specific_layers(x, n):
    y1 = Dropout(prob_h3)(Dense(n, activation=act)(x))
    y2 = Dropout(prob_h3)(Dense(n, activation=act)(x))
    y3 = Dropout(prob_h3)(Dense(n, activation=act)(x))
    y4 = Dropout(prob_h3)(Dense(n, activation=act)(x))
    y5 = Dropout(prob_h3)(Dense(n, activation=act)(x))
    y6 = Dropout(prob_h3)(Dense(n, activation=act)(x))
    y7 = Dropout(prob_h3)(Dense(n, activation=act)(x))
    y8 = Dropout(prob_h3)(Dense(n, activation=act)(x))   
    y9 = Dropout(prob_h3)(Dense(n, activation=act)(x))
    y10 = Dropout(prob_h3)(Dense(n, activation=act)(x))
  
    return y1,y2,y3,y4,y5,y6,y7,y8,y9,y10

def build_model(ni, n0, n1, n2, n3, ln, act):

    # ENTRADA
    model_input = Input(shape=(ni,))
    
    # CAPAS COMUNES A TODAS LOS TARGETS
    x = Dense(n0, activation=act)(model_input)
    # x = BatchNormalization(momentum=momentum_batch_norm)(x)
    x = Dropout(prob_h1)(x)
    x = Dense(n1, activation=act)(x)
    # x = BatchNormalization(momentum=momentum_batch_norm)(x)
    x = Dropout(prob_h2)(x)
    x = Dense(n2, activation=act)(x)
    # x = BatchNormalization(momentum=momentum_batch_norm)(x)
    x = Dropout(prob_h3)(x)
    
    # CAPAS ESPECIFICAS A CADA TARGET
    y1,y2,y3,y4,y5,y6,y7,y8,y9,y10 = specific_layers(x, n3)
    

    # TARGETS 1 a 15
    y1 = Dense(units = 1,activation= 'sigmoid', name = 'output_1')(y1)
    y2 = Dense(units = 1,activation= 'sigmoid', name = 'output_2')(y2)
    y3 = Dense(units = 1,activation= 'sigmoid', name = 'output_3')(y3)
    y4 = Dense(units = 1,activation= 'sigmoid', name = 'output_4')(y4)
    y5 = Dense(units = 1,activation= 'sigmoid', name = 'output_5')(y5)
    y6 = Dense(units = 1,activation= 'sigmoid', name = 'output_6')(y6)
    y7 = Dense(units = 1,activation= 'sigmoid', name = 'output_7')(y7)
    y8 = Dense(units = 1,activation= 'sigmoid', name = 'output_8')(y8)    
    y9 = Dense(units = 1,activation= 'sigmoid', name = 'output_9')(y9)
    y10 =Dense(units = 1,activation= 'sigmoid', name = 'output_10')(y10)

    # MODELO FINAL
    model = Model(inputs=model_input, outputs=[y1,y2,y3,y4,y5,y6,y7,y8,y9,y10])
    
    return model

In [7]:
def run_MTL(x_train, x_val, y_train, y_val, lb = 0.005, arq = (100,50,10,5), w = None):
    
    n_inputs = x_train.shape[1]
    # regularizacion l2 
    reg = l2(lb)
    #optimizador
    adam_opt = Adam(l_rate)
    #early stopping
    early_stop = EarlyStopping(monitor='val_loss',
                               min_delta=min_delta_val,
                               patience=patience_val,
                               verbose=0, 
                               mode='min',
                               restore_best_weights=True)

    model = build_model(n_inputs, arq[0], arq[1], arq[2], arq[3], reg, act)

    #compilar el modelo
    model.compile(loss=BinaryCrossentropy(), optimizer=adam_opt, metrics = [tf.keras.metrics.Precision(),
                                                                            tf.keras.metrics.Recall(),
                                                                            tf.keras.metrics.Accuracy()])
    
    #entrenar el modelo
    learning_data = model.fit(x=x_train,
                              y=y_train,
                              epochs=n_epochs,
                              validation_data=(x_val,y_val),
                              callbacks=[early_stop],
                              class_weight=w,
                              verbose=2)
    
    return model

In [8]:
#------------------------------------------------------------------------------------------------
pesos = {
    '97':{0:1,1:3.47},
    '98':{0:1,1:1.90},
    '100':{0:1,1:1.56},
    '102':{0:1,1:3.31},
    '1535':{0:1,1:5.09},
    '1537':{0:1,1:5.11},
    '1538':{0:1,1:2.81},
    'Extra':{0:1,1:1.10},
    'Overall':{0:1.40,1:1},
    'Bal': {0:1.1,1:1, -1:0} 
}

params_dict = {'a':{'lb':0.005,'arq':(100,50,10,5),'w': None}}
seed_lst = [24]

#------------------------------------------------------------------------------------------------

In [9]:
for seed in seed_lst:
    for idx,split in enumerate(train_folds):
        # aca recuperar los folds y hacer lo de cada fold
        print('hola')
        sys.stdout.flush()
        train,test = split
        x_train, y_train = train
        x_test, y_test = test
        
        # obtener params
        for key in params_dict.keys():
            params = params_dict.get(key)
            lb = params.get('lb')
            arq = params.get('arq')
            w = params.get('w')
            set_seed(seed)
        
            # entrenar modelo
            trained_model = run_MTL(x_train, x_test, y_train, y_test, lb, arq, w)
            
            # computar metricas
            pred = trained_model.evaluate(x_test, y_test)
            # print(pred.shape)
            sys.stdout.flush()
            

hola
Train on 3053 samples, validate on 764 samples
3053/3053 - 92s - loss: 6.8781 - output_1_loss: 0.7015 - output_2_loss: 0.6774 - output_3_loss: 0.6807 - output_4_loss: 0.6859 - output_5_loss: 0.6779 - output_6_loss: 0.6764 - output_7_loss: 0.6992 - output_8_loss: 0.6897 - output_9_loss: 0.6919 - output_10_loss: 0.6971 - output_1_precision: 0.0040 - output_1_recall: 0.6429 - output_1_accuracy: 0.0000e+00 - output_2_precision: 0.0000e+00 - output_2_recall: 0.0000e+00 - output_2_accuracy: 0.0000e+00 - output_3_precision: 0.0000e+00 - output_3_recall: 0.0000e+00 - output_3_accuracy: 0.0000e+00 - output_4_precision: 0.0732 - output_4_recall: 0.0931 - output_4_accuracy: 0.0000e+00 - output_5_precision: 0.0578 - output_5_recall: 0.1189 - output_5_accuracy: 0.0000e+00 - output_6_precision: 0.0582 - output_6_recall: 0.1965 - output_6_accuracy: 0.0000e+00 - output_7_precision: 0.0042 - output_7_recall: 0.4667 - output_7_accuracy: 0.0000e+00 - output_8_precision: 0.0345 - output_8_recall: 0.0